In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt 
import duckdb
import shapely.wkb
from shapely.wkb import loads

import glob
import os

In [2]:
#data files
db = duckdb.read_parquet(r"D:\Datenbank\ohsome-stats-yearly\year=*\month=*\*.parquet", hive_partitioning=1) 

## Data Download

In [3]:
# All corporate Hashtags
hashtag_list = ['amap', 'adt', 'bolt', 'DigitalEgypt', 'expedia', 'gojek', 'MSFTOpenMaps', 'grab', 'Kaart', 'Kontur', 'mbx', 'RocketData',
                'disputed_by_claimed_by', 'Snapp', 'stackbox', 'Telenav', 'Lightcyphers', 'tomtom', 'TIDBO', 'WIGeoGIS-OMV', 'نشان',
                'mapbox', 'Komoot', 'AppLogica']

In [4]:
# Specify the directory where your Excel files are located
directory_path =  r"C:\Users\lilly\Documents\bachelorarbeit\analysis\UserNameID-v2"

# Get a list of all Excel files in the directory
excel_files = glob.glob(os.path.join(directory_path, "*.xls"))

# Create an empty dictionary to store DataFrames
dataframes_dict = {}

# Read each Excel file and store its DataFrame in the dictionary - files saved as xldr - otherwise when saved as csv, other method applicable
for excel_file in excel_files:
    filename = os.path.basename(excel_file)
    df = pd.read_csv(excel_file) 
    dataframes_dict[filename] = df

In [5]:
# get the user-ids in working format
def getListID(filename):
    df = dataframes_dict[filename]
    CorpoId = df['User ID']
    user_ids_str = ','.join([f"'{id}'" for id in CorpoId])
    return user_ids_str

In [6]:
Meta = getListID("MetaUser.xls")

In [7]:
def yearlyContributors(year, filename):
    try: 

        # Construct the CASE statement to group hashtags together
        case_statement = "\n".join([
            f"WHEN hashtags ILIKE '%{hashtag}%' THEN '{hashtag}'" for hashtag in hashtag_list
        ])
        
        # Construct the SQL query
        query = f"""
            SELECT
                CASE
                    {case_statement}
                    ELSE 'nc'
                END AS corporation,
                COUNT(distinct user_id) AS contributors
            FROM db
            WHERE year = {year}
            GROUP BY corporation
            ORDER BY contributors DESC
        """
        
        # Execute the SQL query
        result = duckdb.sql(query)
        
        # Show the result
        hashtags = result.to_df()
    
    
        q2 = f"""
            SELECT COUNT(distinct user_id) AS meta
            FROM db
            WHERE year = {year} AND user_id IN ({Meta})
            
        """
        r2 = duckdb.sql(q2)
        meta = r2.to_df()
    
        m = str(meta['meta'].values[0]) 
        d = {'corporation': ['meta'], 'contributors': [m]}
        meta_df = pd.DataFrame(data=d)
    
        
        frames = [hashtags, meta_df]
    
        table = pd.concat(frames)
    
        table.to_csv(filename, header = True)

    except Exception as e:
        print(f"An error occurred: {e}")

In [11]:
yearlyContributors(2016, "totalContributors_2016.csv")

In [12]:
yearlyContributors(2017, "totalContributors_2017.csv")

In [13]:
yearlyContributors(2018, "totalContributors_2018.csv")

In [8]:
yearlyContributors(2019, "totalContributors_2019.csv")

In [9]:
yearlyContributors(2020, "totalContributors_2020.csv")

In [10]:
yearlyContributors(2021, "totalContributors_2021.csv")

In [11]:
yearlyContributors(2022, "totalContributors_2022.csv")

In [12]:
yearlyContributors(2023, "totalContributors_2023.csv")

## Data Preperation

### Absolute numbers

In [3]:
#extracting  the total corporate and non-corporate information yearly - since the files have saved it per corporation
def extrctingCEdata(csv, yearColumn): 
    df = pd.read_csv(csv)

    df = df.drop(columns={'Unnamed: 0'})
    total = df['contributors'].sum()
    corporate = df['contributors'].tail(-1).sum()
    non_corporate = df['contributors'].head(1).sum()

     #calculating the percentage of corporate edits based on total edits
    percentage = (corporate / total) * 100

    d = {'year' : [yearColumn], 'total': [total], 'CE': [corporate], 'NCE': [non_corporate], 'percentage': [percentage]}

    pd.set_option('display.float_format', '{:.2f}'.format)
    
    df = pd.DataFrame(data=d)

    return df

In [4]:
y16 = extrctingCEdata('totalContributors_2016.csv', 2016)
y17 = extrctingCEdata('totalContributors_2017.csv', 2017)
y18 = extrctingCEdata('totalContributors_2018.csv', 2018)
y19 = extrctingCEdata('totalContributors_2019.csv', 2019)
y20 = extrctingCEdata('totalContributors_2020.csv', 2020)
y21 = extrctingCEdata('totalContributors_2021.csv', 2021)
y22 = extrctingCEdata('totalContributors_2022.csv', 2022)
y23 = extrctingCEdata('totalContributors_2023.csv', 2023)


In [5]:
frames = [y16, y17, y18, y19, y20, y21, y22, y23]

yearly_data = pd.concat(frames)

In [6]:
yearly_data = yearly_data.reset_index().drop(columns={'index'})

In [7]:
yearly_data.to_csv('fig1_2016-2023_yearly_data_ce_nce_contributors.csv', header = True)

### top 10 - contributors per corporation

In [8]:
h16 = pd.read_csv('totalContributors_2016.csv')
h17 = pd.read_csv('totalContributors_2017.csv')
h18 = pd.read_csv('totalContributors_2018.csv')
h19 = pd.read_csv('totalContributors_2019.csv')
h20 = pd.read_csv('totalContributors_2020.csv')
h21 = pd.read_csv('totalContributors_2021.csv')
h22 = pd.read_csv('totalContributors_2022.csv')
h23 = pd.read_csv('totalContributors_2023.csv')

In [9]:
def datprepCE(year):
    df = pd.read_csv(f'totalContributors_{year}.csv')
    df = df.drop(columns={'Unnamed: 0'})
    df = df.rename(columns={'contributors': year})
    return df

In [10]:
years = [2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
dat = {'corporation' : []}
df_t = pd.DataFrame(data=dat)

In [11]:
for year in years:
    d = datprepCE(year)
    df_t = pd.merge(df_t, d, on = 'corporation', how = 'outer')

In [12]:
timeline = df_t.copy()
timeline = timeline.rename_axis('year', axis=1)
timeline = timeline.set_index('corporation')

In [13]:
timeline = timeline.fillna(0)

In [14]:
timeline.loc['mapbox'] += timeline.loc['mbx']

In [15]:
timeline.drop(['mbx'], inplace=True)

In [17]:
edit_count = timeline.transpose().sum()

In [18]:
edit_count.sort_values(ascending = False)

corporation
nc                       2199105.00
amap                        6976.00
adt                         3315.00
Kaart                       1805.00
grab                        1485.00
tomtom                      1100.00
meta                         946.00
mapbox                       515.00
expedia                      392.00
MSFTOpenMaps                 147.00
bolt                         122.00
Telenav                      101.00
DigitalEgypt                  69.00
Lightcyphers                  22.00
نشان                          16.00
stackbox                      16.00
RocketData                    11.00
gojek                         10.00
Kontur                         9.00
Komoot                         7.00
Snapp                          6.00
TIDBO                          5.00
disputed_by_claimed_by         5.00
WIGeoGIS-OMV                   5.00
AppLogica                      2.00
dtype: float64

In [19]:
Other = timeline.copy()
Other = Other.transpose()

In [20]:
Other = Other[['Telenav', 'bolt', 'نشان','RocketData', 'Lightcyphers', 'DigitalEgypt', 'TIDBO', 'Kontur', 'stackbox', 'AppLogica', 'disputed_by_claimed_by', 'WIGeoGIS-OMV', 'gojek', 'Komoot', 'Snapp']]

In [21]:
Other.transpose().sum()

year
2016     0.00
2017     4.00
2018    23.00
2019    55.00
2020    35.00
2021    79.00
2022   115.00
2023    95.00
dtype: float64

In [22]:
large = timeline.transpose().reset_index()
large = large[['year', 'amap', 'adt', 'Kaart', 'grab', 'tomtom', 'meta', 'mapbox', 'expedia', 'MSFTOpenMaps', 'bolt']]

In [23]:
large['Other'] = ['0.0', '4.0', '23.0', '55.0', '35.0', '79.0', '115.0', '95.0']

In [24]:
large['Other'] = large['Other'].apply(pd.to_numeric, errors ='coerce')

### prep for corporation percentage

In [26]:
yearly_data

,year,total,CE,NCE,percentage
0,2016,260242,482,259760,0.19
1,2017,318037,1991,316046,0.63
2,2018,306764,1087,305677,0.35
3,2019,281502,1468,280034,0.52
4,2020,303474,2666,300808,0.88
5,2021,293770,4435,289335,1.51
6,2022,259152,2972,256180,1.15
7,2023,193251,1986,191265,1.03


In [27]:
# calculating the total number of PFs per year - taking the df created for the first graph
newTotal = yearly_data.copy()

totalCE = newTotal[['year', 'CE']]

In [28]:
# adding the total corporate numbers
top10List = pd.merge(totalCE, large, on = 'year', how ='left')

In [29]:
top10List

,year,CE,amap,adt,Kaart,grab,tomtom,meta,mapbox,expedia,MSFTOpenMaps,bolt,Other
0,2016,482,291.00,2.00,28.00,0.00,9.00,1.00,82.00,69.00,0.00,0.00,0.00
1,2017,1991,909.00,21.00,51.00,90.00,640.00,24.00,66.00,186.00,0.00,3.00,4.00
2,2018,1087,364.00,26.00,309.00,150.00,2.00,85.00,22.00,106.00,0.00,0.00,23.00
3,2019,1468,414.00,15.00,446.00,405.00,0.00,124.00,6.00,3.00,0.00,4.00,55.00
4,2020,2666,710.00,1115.00,405.00,116.00,1.00,177.00,61.00,0.00,46.00,7.00,35.00
5,2021,4435,2142.00,1150.00,352.00,322.00,102.00,169.00,72.00,6.00,41.00,26.00,79.00
6,2022,2972,1299.00,573.00,145.00,203.00,274.00,218.00,100.00,11.00,34.00,43.00,115.00
7,2023,1986,847.00,413.00,69.00,199.00,72.00,148.00,106.00,11.00,26.00,39.00,95.00


In [30]:
corpor10 = ['amap','adt', 'Kaart', 'grab', 'tomtom', 'meta', 'mapbox', 'expedia', 'MSFTOpenMaps', 'bolt', 'Other']

In [31]:
# calculating th epercentage each corporation held based on the total corporate edits that year
newDF = top10List
def calc_perc(hashtag):
    newDF = top10List
    newDF[f'{hashtag}_perc'] =  newDF[hashtag]/newDF['CE'] *100
    newDF = newDF.drop(columns={f'{hashtag}'})
    return newDF

In [32]:
for tags in corpor10: 
    calc_perc(tags)

In [33]:
newDF

,year,CE,amap,adt,Kaart,grab,tomtom,meta,mapbox,expedia,...,adt_perc,Kaart_perc,grab_perc,tomtom_perc,meta_perc,mapbox_perc,expedia_perc,MSFTOpenMaps_perc,bolt_perc,Other_perc
0,2016,482,291.00,2.00,28.00,0.00,9.00,1.00,82.00,69.00,...,0.41,5.81,0.00,1.87,0.21,17.01,14.32,0.00,0.00,0.00
1,2017,1991,909.00,21.00,51.00,90.00,640.00,24.00,66.00,186.00,...,1.05,2.56,4.52,32.14,1.21,3.31,9.34,0.00,0.15,0.20
2,2018,1087,364.00,26.00,309.00,150.00,2.00,85.00,22.00,106.00,...,2.39,28.43,13.80,0.18,7.82,2.02,9.75,0.00,0.00,2.12
3,2019,1468,414.00,15.00,446.00,405.00,0.00,124.00,6.00,3.00,...,1.02,30.38,27.59,0.00,8.45,0.41,0.20,0.00,0.27,3.75
4,2020,2666,710.00,1115.00,405.00,116.00,1.00,177.00,61.00,0.00,...,41.82,15.19,4.35,0.04,6.64,2.29,0.00,1.73,0.26,1.31
5,2021,4435,2142.00,1150.00,352.00,322.00,102.00,169.00,72.00,6.00,...,25.93,7.94,7.26,2.30,3.81,1.62,0.14,0.92,0.59,1.78
6,2022,2972,1299.00,573.00,145.00,203.00,274.00,218.00,100.00,11.00,...,19.28,4.88,6.83,9.22,7.34,3.36,0.37,1.14,1.45,3.87
7,2023,1986,847.00,413.00,69.00,199.00,72.00,148.00,106.00,11.00,...,20.80,3.47,10.02,3.63,7.45,5.34,0.55,1.31,1.96,4.78


In [34]:
percCE = newDF.drop(columns={'CE','meta', 'adt', 'Kaart', 'amap', 'MSFTOpenMaps', 'tomtom', 'grab', 'expedia', 'bolt', 'mapbox', 'Other'})

In [35]:
percCE = percCE.rename(columns = {'amap_perc': 'amap',
                                  'meta_perc': 'meta',
                                  'Kaart_perc':'Kaart',
                                  'tomtom_perc':'tomtom',
                                  'adt_perc': 'adt', 
                                  'grab_perc': 'grab',
                                  'bolt_perc':'bolt', 
                                  'MSFTOpenMaps_perc':'MSFTOpenMaps',
                                  'expedia_perc':'expedia',
                                  'bolt_perc': 'bolt',
                                  'Other_perc':'Other',
                                  'mapbox_perc': 'mapbox'
                                 })

In [36]:
percCE

,year,amap,adt,Kaart,grab,tomtom,meta,mapbox,expedia,MSFTOpenMaps,bolt,Other
0,2016,60.37,0.41,5.81,0.00,1.87,0.21,17.01,14.32,0.00,0.00,0.00
1,2017,45.66,1.05,2.56,4.52,32.14,1.21,3.31,9.34,0.00,0.15,0.20
2,2018,33.49,2.39,28.43,13.80,0.18,7.82,2.02,9.75,0.00,0.00,2.12
3,2019,28.20,1.02,30.38,27.59,0.00,8.45,0.41,0.20,0.00,0.27,3.75
4,2020,26.63,41.82,15.19,4.35,0.04,6.64,2.29,0.00,1.73,0.26,1.31
5,2021,48.30,25.93,7.94,7.26,2.30,3.81,1.62,0.14,0.92,0.59,1.78
6,2022,43.71,19.28,4.88,6.83,9.22,7.34,3.36,0.37,1.14,1.45,3.87
7,2023,42.65,20.80,3.47,10.02,3.63,7.45,5.34,0.55,1.31,1.96,4.78


### percentage contributors per corporation based on total contributors
same as above but instead of the 'CE' column in totalCE  its now the 'total' column in total edits

In [52]:
# calculating the total number of PFs per year - taking the df created for the first graph
newTotal = yearly_data.copy()

total_contributors = newTotal[['year', 'total']]

In [53]:
# adding the total corporate numbers
top10List_new = pd.merge(total_contributors, large, on = 'year', how ='left')

In [54]:
corpor10 

['amap',
 'adt',
 'Kaart',
 'grab',
 'tomtom',
 'meta',
 'mapbox',
 'expedia',
 'MSFTOpenMaps',
 'bolt',
 'Other']

In [47]:
# adjusted for total column
newDF_total = top10List_new
def calc_perc_edits(hashtag):
    newDF_total = top10List_new
    newDF_total[f'{hashtag}_perc'] =  newDF_total[hashtag]/newDF_total['total'] *100
    newDF_total = newDF_total.drop(columns={f'{hashtag}'})
    return newDF_total

In [48]:
for tags in corpor10: 
    calc_perc_edits(tags)

In [57]:
perc_total = newDF_total.drop(columns={'total','meta', 'adt', 'Kaart', 'amap', 'MSFTOpenMaps', 'tomtom', 'grab', 'expedia', 'mapbox', 'bolt', 'Other'})

In [58]:
perc_total = perc_total.rename(columns = {'amap_perc': 'amap',
                                  'meta_perc': 'meta',
                                  'Kaart_perc':'Kaart',
                                  'tomtom_perc':'tomtom',
                                  'adt_perc': 'adt', 
                                  'grab_perc': 'grab',
                                  'bolt_perc':'bolt', 
                                  'MSFTOpenMaps_perc':'MSFTOpenMaps',
                                  'expedia_perc':'expedia',
                                  'bolt_perc': 'bolt',
                                  'Other_perc':'Other',
                                  'mapbox_perc': 'mapbox'
                                 })

In [59]:
perc_total

,year,amap,adt,Kaart,grab,tomtom,meta,mapbox,expedia,MSFTOpenMaps,bolt,Other
0,2016,0.11,0.00,0.01,0.00,0.00,0.00,0.03,0.03,0.00,0.00,0.00
1,2017,0.29,0.01,0.02,0.03,0.20,0.01,0.02,0.06,0.00,0.00,0.00
2,2018,0.12,0.01,0.10,0.05,0.00,0.03,0.01,0.03,0.00,0.00,0.01
3,2019,0.15,0.01,0.16,0.14,0.00,0.04,0.00,0.00,0.00,0.00,0.02
4,2020,0.23,0.37,0.13,0.04,0.00,0.06,0.02,0.00,0.02,0.00,0.01
5,2021,0.73,0.39,0.12,0.11,0.03,0.06,0.02,0.00,0.01,0.01,0.03
6,2022,0.50,0.22,0.06,0.08,0.11,0.08,0.04,0.00,0.01,0.02,0.04
7,2023,0.44,0.21,0.04,0.10,0.04,0.08,0.05,0.01,0.01,0.02,0.05


In [60]:
perc_total.to_csv('fig1_top10_corpos_percentage_contributors_based_on_total_contributors.csv', header = True)